In [ ]:
#####################################################################################
############## GEDI DATA DOWNLOAD AND INFORMATION: EXAMPLES #########################
############## NEHA HUNKA, UNIVERSITY OF MARYLAND ###################################
#####################################################################################

# Install Packages - only need to do this once
install.packages(c("rstac", "terra", "RColorBrewer", "getPass", "earthdatalogin"))
install.packages("plotly")
if (!require("BiocManager", quietly = TRUE))
  install.packages("BiocManager")
BiocManager::install("rhdf5")
devtools::install_git("https://github.com/carlos-alberto-silva/rGEDI", dependencies = TRUE)

library(rhdf5)
library(httr)
library(purrr)
library(stringr)
library(sf)
library(rGEDI)
library(ggplot2)
library(plotly)
library(curl)
library(rstac)          # interact with stac catalogs
library(terra)          # work with raster data
library(RColorBrewer)   # color pallets from Color Brewer, a library of colors designed for optimized readability in cartograhy
library(getPass)        # allows us to give the code our password without it printing to the console
library(earthdatalogin) # lets us authenticate to access NASA data

# Set your working directory
setwd("/Users/nhunka/Documents/Biomass_Harmonization/Data/Senegal_Workshop/R_Codes")
source("/Users/nhunka/Documents/Biomass_Harmonization/Data/Senegal_Workshop/R_Codes/GEDI_FUN.R")

#####################################################################
######## FIRST, LET'S LEARN HOW TO AUTOMATE DOWNLOADS FROM EARTHDATA
#####################################################################

## Set your output folder 
folder = "/Users/nhunka/Documents/Biomass_Harmonization/Data/Senegal_Workshop/Data/L2A/"

## Read the area of interest (AOI) for which you want GEDI data
AOI = ("/Users/nhunka/Documents/Biomass_Harmonization/Data/Senegal_Workshop/GGNP_Nigeria.gpkg")
AOI_vect = st_read(AOI)
AOI_bbox = st_bbox(AOI_vect)
# We need a bounding-box defined for STAC queries
# minimum longitude, minimum latitude, maximum longitude, and maximum latitude 
bbox = c(AOI_bbox[2],AOI_bbox[4],AOI_bbox[1],AOI_bbox[3])
# Later we'll need the same bbox as a Terra extent object for reading data
extent = ext(c(AOI_bbox[1],AOI_bbox[3],AOI_bbox[2],AOI_bbox[4]))
plot(AOI_vect[1], col="lightblue") #ok, this is perhaps not that

# GDAL optimization for working with cloud-optimized data
setGDALconfig("GDAL_DISABLE_READDIR_ON_OPEN", "EMPTY_DIR")
setGDALconfig("CPL_VSIL_CURL_ALLOWED_EXTENSIONS", "TIF") 
earthdata_username <- getPass("Earthdata User:")
earthdata_password <- getPass("Earthdata Password:")

# note: adding forcemask=FALSE would let you see the values as you type. We intentionally hid that for security of the presentation.
earthdatalogin::edl_netrc(username = earthdata_username, password = earthdata_password)

#####################################################################
########## NOW WE START A QUERY ON EARTH DATA FOR GEDI L2A FILES ####
#####################################################################

# connect to the stac endpoint we want to query
nasa_stac <- stac("https://cmr.earthdata.nasa.gov/stac/LPCLOUD")

search_hls <- stac_search(
  q = nasa_stac, # The STAC API connection we made earlier
  collections = "GEDI02_A_002", # https://lpdaac.usgs.gov/products/hlss30v002/
  bbox = bbox, #bounding box that we made eariler
  datetime = "2022-06-01T00:00:00Z/2022-07-30T00:00:00Z", 
  limit = 100 #limits how many results we see
)
results_hls <- post_request(search_hls)
urls<-assets_url(results_hls)
urls<-urls[grep('.h5', urls)]
setwd(folder)

#####################################################################
######## ONCE WE HAVE URLS OF THE FILES, LAUNCH DOWNLOADS ###########
#####################################################################
setwd("/Users/nhunka/Documents/Biomass_Harmonization/Data/Senegal_Workshop/Data/L2A")
edl_netrc(earthdata_username, earthdata_password, "netrc") #only need to do this once
for (url in urls[58]){
  if (!file.exists(paste0(folder,basename(url)))){
    skip <- FALSE
    tryCatch(
      edl_download(url, netrc_path = "netrc",cookie_path = edl_cookie_path(), quiet = FALSE,method="curl"),
      error = function(e) { skip <<- TRUE})
    if(skip) { next } 
  }
}

##################################################################
############ READ ALL H5 FILES AND CONVERT TO GPKG ###############
#####################################################################

crs    <- CRS("+init=epsg:4326") 
L2A.ls<-list.files(folder, recursive = TRUE, full.names = TRUE)
L2A.ls<-L2A.ls[str_detect(L2A.ls,pattern=".h5")]
L2A.ls
for (h5_file in L2A.ls[10:19]){
  if (!file.exists(paste0(folder,gsub(".h5",".gpkg",  basename(h5_file))))){
    skip <- FALSE
    tryCatch(
      l <- readLevel2A(h5_file),
      L2A.m <- getLevel2A(l),
      L2A.sp <- SpatialPointsDataFrame(coords=cbind(L2A.m$lon_lowestmode,L2A.m$lat_lowestmode),data=L2A.m,proj4string = crs),
      L2A.sp <- st_as_sf(L2A.sp),
      L2A.sp <- st_intersection(L2A.sp,AOI_vect),
      L2A.sp <- L2A.sp[L2A.sp$quality_flag == 1,],
      L2A.sp <- L2A.sp[L2A.sp$sensitivity > 0.95,],
      if (nrow(L2A.sp) > 0) {st_write(L2A.sp, paste0(folder,gsub(".h5",".gpkg",  basename(h5_file))))}, 
      error = function(e) { skip <<- TRUE})
    if(skip) { next } 
  }
}

#####################################################################
##################### MERGE ALL GPKGs INTO ONE FOR OUR AOI ##########
#####################################################################

gpkg_files <- list.files(folder, pattern = "\\.gpkg$", full.names = TRUE)
gpkg_files
sf_list <- list()
for (file in gpkg_files) {
  skip <- FALSE
  tryCatch(
    sf_data <- st_read(file),
    sf_list[[length(sf_list) + 1]] <- sf_data,
    error = function(e) { skip <<- TRUE})
  if(skip) { next } 
}
merged_sf <- do.call(rbind, sf_list)
st_write(merged_sf, paste0(folder,gsub(".gpkg","_GEDI_L2A.gpkg",  basename(AOI))))

####################################################################################
############################# MAKE A GRIDDED MAP OF YOUR DATA ######################
####################################################################################
setwd("/Users/nhunka/Documents/Biomass_Harmonization/Data/Senegal_Workshop/Data")

gpkg_file <- "GGNP_Nigeria_SUD_L2A.gpkg"  # Specify the path to your GeoPackage file
layer_name <- "GGNP_Nigeria_SUD_L2A"         # Specify the layer name you want to rasterize
cell_size <- 500                        # Specify the desired cell size (in map units, e.g., meters)
target_crs <- "EPSG:6933"               # Specify the target CRS (projection), e.g., WGS84

# Read the vector data from the GeoPackage
vector_data <- st_read(gpkg_file, layer = layer_name)

# Ensure the vector data is in the correct CRS (transform if necessary)
vector_data <- st_transform(vector_data, crs = target_crs)

# Create an empty raster with the desired cell size and extent based on the vector data
raster_extent <- ext(st_bbox(vector_data)) # Get the bounding box from the vector data
raster_template <- rast(raster_extent, resolution = cell_size)

# Rasterize the vector data
rasterized_data <- rasterize(vector_data, raster_template, field = "rh98",fun="mean")
plot(rasterized_data,zlim=c(0,200))

# Save the rasterized result to a file
output_raster_file <- "rasterized_output.tif"  # Specify the output raster file path
writeRaster(rasterized_data, output_raster_file, overwrite = TRUE)

ggplot() + coord_sf(crs = 6933) + 
  geom_point(aes(x = loc.plot[,1], y = loc.plot[,2], col = agbd.plot),size=0.5) +  # make a map to see plot locations and the AGBD values at each location
  scale_color_viridis(limits = c(0, 150)) 

r_df <- as.data.frame(rasterized_data, xy = TRUE, na.rm = TRUE)

# Create the plot using ggplot2 with limits on the color scale
ggplot(r_df, aes(x = x, y = y, fill = mean)) + 
  geom_raster() + 
  scale_fill_viridis(option = "D", limits = c(0, 40)) +  # Set limits for the color scale
  theme_minimal() +
  theme(legend.position = "right") +
  labs(fill = "GEDI RH98", title = "GEDI Height Map", x = "Longitude", y = "Latitude")

####################################################################################
############################# MAKE A 3D PLOT OF THE L2A DATASET ####################
####################################################################################
bbox <- c(xmin = 11.50720, ymin = 7.29186, xmax = 11.53329, ymax = 7.31127) 
bbox <- c(xmin = 11.46186, ymin = 7.25631, xmax = 11.51621, ymax = 7.30357)

vector_data <- vector_data %>% st_transform("EPSG:4326")
vector_subset <- st_crop(vector_data, bbox)

# Define different columns for overplotting
x1 <- vector_subset$lon_lowestmode
y1 <- vector_subset$lat_lowestmode

# Create the 3D scatter plot with multiple overlaid traces
plot_ly() %>%
  add_trace(x = x1, y = y1, z = vector_subset$rh20, type = 'scatter3d', mode = 'markers', 
            marker = list(color = 'yellow', size = 5, opacity = 0.7), 
            name = 'RH20') %>%
  add_trace(x = x1, y = y1, z = vector_subset$rh50, type = 'scatter3d', mode = 'markers', 
            marker = list(color = 'blue', size = 5, opacity = 0.7), 
            name = 'RH50') %>%
  add_trace(x = x1, y = y1, z = vector_subset$rh98, type = 'scatter3d', mode = 'markers',
            marker = list(color = 'green', size = 5, opacity = 0.7),
            name = 'RH98') %>%
  layout(scene = list(
    xaxis = list(title = 'Longitude'),
    yaxis = list(title = 'Latitude'),
    zaxis = list(title = 'Heights')
  ),
  title = '3D Scatter Plot with Multiple Overplotted Data Series')
